In [1]:
import pandas as pd

weather_data = pd.read_csv("../Data/daily_weatherdata_2022.csv", header = 0, sep=',')
print(weather_data.head())

cutoff_rain_day = 0.0002

   Month  Day  LC_RAININ  LC_DAILYRAIN  LC_WINDDIR  LC_WINDSPEED       Hour   
0      1    1   0.000002      0.000043   -6.263419      0.324702  11.491857  \
1      1    2   0.000671      0.003387  -28.223994      0.707815  11.500000   
2      1    3   0.000729      0.006724  -34.624291      0.650430  11.500000   
3      1    4   0.000454      0.003578  -25.189413      0.326204  11.500000   
4      1    5   0.000127      0.000255  -43.943098      0.602983  11.500000   

   LC_TEMP_QCL3  
0     12.194145  
1     11.893159  
2      9.539620  
3      6.768886  
4      3.889968  


In [26]:
weather_data["bool_rainday"] = weather_data["LC_DAILYRAIN"] > cutoff_rain_day

weather_data.head()

,Month,Day,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Hour,LC_TEMP_QCL3,bool_rainday
0,1,1,0.000002,0.000043,-6.263419,0.324702,11.491857,12.194145,False
1,1,2,0.000671,0.003387,-28.223994,0.707815,11.500000,11.893159,True
2,1,3,0.000729,0.006724,-34.624291,0.650430,11.500000,9.539620,True
3,1,4,0.000454,0.003578,-25.189413,0.326204,11.500000,6.768886,True
4,1,5,0.000127,0.000255,-43.943098,0.602983,11.500000,3.889968,True


In [4]:
# Group the data by "Month" and count the number of True values in "bool_rainday" per month
rainy_days_per_month = weather_data.groupby("Month")["bool_rainday"].sum()

rainy_days_per_month.head()

Month
1    13
2    16
3     2
4     8
5     8
Name: bool_rainday, dtype: int64

In [5]:
data_month = pd.read_csv('../Data/monthly_weatherdata_2022.csv')
data_month.head()

,Month,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Day,Hour,LC_TEMP_QCL3
0,1,0.000112,0.001033,-16.334089,0.336535,15.999657,11.499737,4.698144
1,2,0.000131,0.001257,-25.171683,0.740747,14.500000,11.500000,6.923138
2,3,0.000010,0.000105,12.190264,0.250732,16.000000,11.500000,8.113201
3,4,0.000054,0.000503,3.643447,0.369271,15.500311,11.500246,10.704171
4,5,0.000076,0.000653,-9.022240,0.240475,16.000000,11.500000,15.572697


In [30]:
import plotly.graph_objects as go

months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
max_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# Create an empty list to store the subplot figures
subplot_figures = []

# Iterate over each month
for i, month in enumerate(months):
    i = i + 1

    # Filter the weather data for the current month
    month_data = weather_data[weather_data["Month"] == i]
    
    # Create a subplot figure for the rain data
    fig = go.Figure()
    
    # Add the rain squares to the figure
    for day in range(1, max_days[i - 1] + 1):  # Iterate up to max_days[i-1] + 1 to include the last day
        
        is_rainy = month_data[month_data["Day"] == day]["bool_rainday"].values[0]
        color = "blue" if is_rainy else "yellow"
        fig.add_shape(type="rect", x0=day - 0.5, y0=0, x1=day + 0.5, y1=1,
                      fillcolor=color, line=dict(color="white"))
    
    # Set the layout for the rain figure
    fig.update_layout(title=f"Rain in {month}",
                      xaxis=dict(range=[0, max_days[i - 1] + 1], showgrid=False, zeroline=False, showticklabels=False),
                      yaxis=dict(range=[0, 1], showgrid=False, zeroline=False, showticklabels=False),
                      height=100)
    
    # Append the subplot figure to the list
    subplot_figures.append(fig)

# Create an empty list to store the subplots
subplots = []

# Iterate over each month and create the subplot
for i, month in enumerate(months):
    subplot = go.Scatter(x=[0], y=[0], mode="markers", marker=dict(color="white"),
                         name=month, legendgroup=month, showlegend=False)
    
    # Set the position and size of the subplot within the main figure
    subplot['xaxis'] = f"x{i+1}"
    subplot['yaxis'] = "y2"
    
    # Add the subplot to the list
    subplots.append(subplot)

# Add the average temperature and windspeed columns
subplots.append(go.Scatter(x=months, y=data_month["LC_TEMP_QCL3"], mode="lines+markers", name="Avg. Temperature"))
subplots.append(go.Scatter(x=months, y=data_month["LC_WINDSPEED"], mode="lines+markers", name="Avg. Windspeed"))

# Set the layout for the main figure
layout = go.Layout(title="Monthly Weather Summary",
                   xaxis=dict(showgrid=False, zeroline=False, tickmode="array", tickvals=list(range(len(months))), ticktext=months),
                   yaxis=dict(showgrid=False, zeroline=False),
                   yaxis2=dict(showgrid=False, zeroline=False, domain=[0.4, 0.6]))

# Create the main figure with subplots
fig = go.Figure(data=subplots, layout=layout)

# Set the subplot figures as annotations within the main figure
for i, subplot_fig in enumerate(subplot_figures):
    fig.add_annotation(xref="paper", yref="paper", x=0.98, y=0.98, xanchor="right", yanchor="top",
                       text=f"Rain in {months[i]}", showarrow=False, font=dict(size=10),
                       bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="rgba(0, 0, 0, 0.3)")
    for trace in subplot_fig['data']:
        fig.add_trace(trace)

# Show the final figure
fig.show()

In [33]:
import plotly.graph_objects as go
import numpy as np

# Filter the weather data for the month of January
january_data = weather_data[weather_data["Month"] == 1]

# Create an empty matrix to store the rain data
matrix = np.zeros((1, max_days[0]))

# Iterate over each day in January
for day in range(1, max_days[0] + 1):
    is_rainy = january_data[january_data["Day"] == day]["bool_rainday"].values[0]
    matrix[0, day - 1] = 1 if is_rainy else 0

# Create the heatmap figure
fig = go.Figure(data=go.Heatmap(z=matrix, colorscale=[[0, 'yellow'], [1, 'blue']],
                               x=list(range(1, max_days[0] + 1)), y=["January"],
                               showscale=False))

# Set the layout for the heatmap
fig.update_layout(title="Rain in January",
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=True))

# Show the heatmap
fig.show()


In [54]:
import plotly.graph_objects as go
import numpy as np

# Filter the weather data for the month of January
january_data = weather_data[weather_data["Month"] == 1]

# Create an empty matrix to store the rain data
matrix = np.zeros((1, max_days[0]))

# Iterate over each day in January
for day in range(1, max_days[0] + 1):
    is_rainy = january_data[january_data["Day"] == day]["bool_rainday"].values[0]
    matrix[0, day - 1] = 1 if is_rainy else 0

# Calculate the width and height of each cell in the heatmap
cell_width = 1
cell_height = 10 / len(matrix)

# Create the heatmap figure
fig = go.Figure(data=go.Heatmap(z=matrix, colorscale=[[0, 'red'], [1, 'blue']],
                               x=list(range(1, max_days[0] + 1)), y=["January"],
                               showscale=False,xgap=1))

# Set the layout for the heatmap
fig.update_layout(title="Rain in January",
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=True,
                             range=[-0.5,0.5]),
                  height=250)

# Show the heatmap
fig.show()


In [65]:
import plotly.graph_objects as go
import plotly.subplots as sp
import numpy as np

# Define the months and maximum days
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
max_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# Create an empty list to store the subplot figures
subplot_figures = []

# Iterate over each month
for month_num, month_name in enumerate(months):
    # Filter the weather data for the current month
    month_data = weather_data[weather_data["Month"] == month_num + 1]

    # Create an empty matrix to store the rain data
    matrix = np.zeros((1, max_days[month_num]))

    # Iterate over each day in the month
    for day in range(1, max_days[month_num] + 1):
        is_rainy = month_data[month_data["Day"] == day]["bool_rainday"].values[0]
        matrix[0, day - 1] = 1 if is_rainy else 0

    # Calculate the width and height of each cell in the heatmap
    cell_width = 1
    cell_height = 1 / len(matrix)

    # Create the heatmap figure
    fig = go.Figure(data=go.Heatmap(z=matrix, colorscale=[[0, 'red'], [1, 'blue']],
                                   x=list(range(1, max_days[month_num] + 1)), y=[month_name],
                                   showscale=False, xgap=1, ygap=0.1))

    # Set the layout for the heatmap
    fig.update_layout(title="", #f"Rain in {month_name}",
                      xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
                      yaxis=dict(showgrid=False, zeroline=False, showticklabels=True,
                                 range=[-0.5, 0.5]),
                      height=50)

    # Append the subplot figure to the list
    subplot_figures.append(fig)

# Create a figure with subplots
fig = sp.make_subplots(rows=len(months), cols=1, shared_xaxes=True, vertical_spacing=0.05)   #, subplot_titles=months

# Add the heatmap subplots to the figure
for i, subplot_fig in enumerate(subplot_figures):
    fig.add_trace(subplot_fig.data[0], row=i + 1, col=1)

# Update the subplot layout
fig.update_layout(height=60 * len(months), title="Monthly Rain Heatmaps")

# Show the figure
fig.show()

In [74]:
import plotly.graph_objects as go
import plotly.subplots as sp
import numpy as np

# Define the months and maximum days
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
max_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# Create an empty list to store the subplot figures
subplot_figures = []

# Iterate over each month
for month_num, month_name in enumerate(months):
    # Filter the weather data for the current month
    month_data = weather_data[weather_data["Month"] == month_num + 1]

    # Create an empty matrix to store the rain data
    matrix = np.zeros((1, max_days[month_num]))

    # Iterate over each day in the month
    for day in range(1, max_days[month_num] + 1):
        is_rainy = month_data[month_data["Day"] == day]["bool_rainday"].values[0]
        matrix[0, day - 1] = 1 if is_rainy else 0

    # Calculate the width and height of each cell in the heatmap
    cell_width = 1
    cell_height = 1 / len(matrix)

    color_zero = "red"
    color_one = "blue"

    #handle extreme cases where all dry or wet
    if np.all(matrix == 1):
        color_zero = "blue"
    elif np.all(matrix == 1):
        color_one = "red"

    # Create the heatmap figure
    fig = go.Figure(data=go.Heatmap(z=matrix, colorscale=[[0, color_zero], [1, color_one]],
                                   x=list(range(1, max_days[month_num] + 1)), y=[month_name],
                                   showscale=False, xgap=1, ygap=0.1))

    # Set the layout for the heatmap
    fig.update_layout(title="", #f"Rain in {month_name}",
                      xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
                      yaxis=dict(showgrid=False, zeroline=False, showticklabels=True,
                                 range=[-0.5, 0.5]),
                      height=50)

    # Append the subplot figure to the list
    subplot_figures.append(fig)

# Create a figure with subplots
fig = sp.make_subplots(rows=len(months), cols=1, shared_xaxes=True, vertical_spacing=0.05)   #, subplot_titles=months

# Add the heatmap subplots to the figure
for i, subplot_fig in enumerate(subplot_figures):
    fig.add_trace(subplot_fig.data[0], row=i + 1, col=1)

# Update the subplot layout
fig.update_layout(height=60 * len(months), title="Monthly Rain Heatmaps")

# Show the figure
fig.show()

Heatmap({
    'colorscale': [[0, 'blue'], [1, 'blue']],
    'showscale': False,
    'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
          21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
    'xgap': 1,
    'y': [December],
    'ygap': 0.1,
    'z': array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
})


In [71]:
print(weather_data[weather_data["Month"]==12])

print(weather_data[weather_data["Month"]==12].head())

     Month  Day  LC_RAININ  LC_DAILYRAIN  LC_WINDDIR  LC_WINDSPEED  Hour   
334     12    1   0.000011      0.001454    4.673997      0.111588  11.5  \
335     12    2   0.000010      0.001454   16.512281      0.321917  11.5   
336     12    3   0.000009      0.001454   22.371914      0.277401  11.5   
337     12    4   0.000016      0.001454   20.623907      0.221765  11.5   
338     12    5   0.000356      0.006575    1.239905      0.015332  11.5   
339     12    6   0.000046      0.001502  -12.479102      0.066309  11.5   
340     12    7   0.000070      0.001579  -24.214185      0.136233  11.5   
341     12    8   0.000034      0.001548  -26.588799      0.239298  11.5   
342     12    9   0.000023      0.001433    0.080376      0.034289  11.5   
343     12   10   0.000019      0.001426  -20.555941      0.179580  11.5   
344     12   11   0.000022      0.001463    1.569573      0.112499  11.5   
345     12   12   0.000019      0.001426    6.950103      0.048556  11.5   
346     12  